# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [2]:
import pandas as pd

## imports for Python, Pandas

In [3]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [4]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [5]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [6]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [7]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{'_id': {'$oid': '52b213b38594d8a2be17c780'},
  'approvalfy': 1999,
  'board_approval_month': 'November',
  'boardapprovaldate': '2013-11-12T00:00:00Z',
  'borrower': 'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  'closingdate': '2018-07-07T00:00:00Z',
  'country_namecode': 'Federal Democratic Republic of Ethiopia!$!ET',
  'countrycode': 'ET',
  'countryname': 'Federal Democratic Republic of Ethiopia',
  'countryshortname': 'Ethiopia',
  'docty': 'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  'envassesmentcategorycode': 'C',
  'grantamt': 0,
  'ibrdcommamt': 0,
  'id': 'P129828',
  'idacommamt': 130000000,
  'impagency': 'MINISTRY OF EDUCATION',
  'lendinginstr': 'Investment Project Financing',
  'lendinginstrtype': 'IN',
  'lendprojectcost': 550000000,
  'majorsector_percent': [{'Name': 'Education', 'Percent': 46},
   {'Name': 'Education', 'Percent': 26},
   {'Name': 'Public Administration, Law, and Justice', 'Percent': 16},
   {'Name': 'Educatio

In [33]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [56]:
json_df = pd.read_json('data/world_bank_projects.json').set_index('id')

In [607]:
projs_by_country = json_df[json_df.countrycode.str.isalpha()]
country_n_projs = pd.DataFrame(
    projs_by_country
    .groupby('countryshortname')
    .count().countryname.reset_index()
    .sort_values(['countryname','countryshortname'], ascending=[False, True])
    .reset_index(drop=True))
country_n_projs['rank'] = country_n_projs.index+1
country_n_projs.columns = ['country', 'n_projects', 'rank']
country_ranks = (country_n_projs
                 .sort_values('rank', ascending=True)
                 .groupby(['n_projects', 'country']).sum().reset_index()
                 .sort_values(['n_projects', 'rank'], ascending=[False, True])
                 .set_index(['n_projects', 'rank']))

In [608]:
country_ranks[:10]

country
n_projects rank                    
19         1                  China
           2              Indonesia
17         3                Vietnam
16         4                  India
13         5     Yemen, Republic of
12         6             Bangladesh
           7                Morocco
           8                  Nepal
11         9             Mozambique
9          10                Brazil

In [676]:
from collections import Counter
from collections import defaultdict

n_theme_counts = Counter((
    json_df.theme_namecode
    .dropna().apply(
        lambda x: [(d.get('code'), d.get('name')) for d in x]))
    .sum()).most_common(10)
n_mjthemes = (pd.DataFrame([
    dict(code=d[0][0], theme=d[0][1], n_count=d[1]) for d in n_theme_counts]))
n_mjthemes['rank'] = n_mjthemes.index+1
n_mjthemes.set_index(['rank'])[['code', 'theme', 'n_count']]

,code,theme,n_count
rank,,,
1,78,Rural services and infrastructure,90
2,27,"Public expenditure, financial management and p...",68
3,81,Climate change,64
4,39,Infrastructure services for private sector dev...,55
5,57,Participation and civic engagement,48
6,79,Other rural development,44
7,52,Natural disaster management,42
8,59,Gender,40
9,65,Education for all,39


In [611]:
codebook = dict(json_df
 .theme_namecode.dropna()
 .apply(lambda x: [(d.get('code'),d.get('name')) for d in x]).sum())

top_themes = pd.DataFrame(
    data=Counter(
        json_df.theme_namecode
        .dropna()
        .apply(lambda x: [d.get('code') for d in x])
        .sum()).most_common(10),
    columns=['mjtheme_code', 'n_count'])
top_themes['rank'] = top_themes.index+1
top_themes['theme'] = top_themes.mjtheme_code.apply(lambda k: codebook[str(k)])
top_mjthemes = top_themes.set_index('rank')[['theme', 'mjtheme_code', 'n_count']]

In [612]:
top_mjthemes

,theme,mjtheme_code,n_count
rank,,,
1,Rural services and infrastructure,78,90
2,"Public expenditure, financial management and p...",27,68
3,Climate change,81,64
4,Infrastructure services for private sector dev...,39,55
5,Participation and civic engagement,57,48
6,Other rural development,79,44
7,Natural disaster management,52,42
8,Gender,59,40
9,Education for all,65,39
